In [51]:
### PROGRAM TO SIZE AND EVALUATE THE  PERFORMANCE OF RAINWATER HARVESTING SYSTEMS (RWHS) 
##Input data: sizing and economic parameters file, DUs files (tariff, costs and discount rate), rainfall file and demand file.
##Obs: results files are created by the program and saved to the folder where that script is saved.

import csv, math
from math import ceil
import numpy as np
from decimal import Decimal
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta

#OPEN INPUT FILES

##GENERAL DATA FILE - OPEN
data_file=('Data - Serrolandia53') #input file name with sizing and economic parameters
with open (data_file+'.csv', 'r') as file:
    d_file=csv.reader(file,delimiter=';') #reads the file delimited by ";"
    rows=list(d_file) #turns the file into a list
file.close()

##SIZING DATA
teta=Decimal(rows[24][0]) #theta to define whether YBS or YAS
c_runoff=Decimal(rows[24][1]) #surface runoff coef
discard_ini=Decimal(rows[24][2]) #initial discard
efficiency=Decimal(rows[24][3]) #desired efficiency
date0=datetime.strptime(rows[24][4],'%d/%m/%Y') #initial date

##CATCHMENT AREAS DATA
###Make a list of areas
total_areas=Decimal(rows[24][5])
area_counter=0 #areas counter
l_areas=[] #area per consumption range
while area_counter<2*total_areas: #while the counter is less than the considered qty
    l_areas.append(Decimal(rows[27][area_counter+1])) #makes a list with the areas per consumption range
    area_counter+=2
    
##CONNECTIONS DATA
###Make a list of connections
total_connections=Decimal(rows[24][5])
connections_counter=0 #connections counter
l_connections=[] #connection per consumption range
while connections_counter<2*total_connections: #while the counter is less than the considered qty
    l_connections.append(Decimal(rows[30][connections_counter+1])) #makes a list with the connections per consumption range
    connections_counter+=2

##ECONOMIC DATA
cmonthlyf=Decimal(rows[36][0]) #fixed monthly cost
cmonthlyv=Decimal(rows[36][1]) #variable monthly cost with the rainwater consumed volume
cquarterly=Decimal(rows[36][2]) #fixed quarterly cost 
csemester=Decimal(rows[36][3]) #fixed semester cost
cannualf=Decimal(rows[36][4]) #fixed annual cost
cannualv=Decimal(rows[36][5]) #variable annual cost with the initial investment
lifespan=Decimal(rows[40][0]) #lifespan
td=Decimal(rows[40][1]) #discount rate (% per month)
expen_m=Decimal(rows[40][2]) #expenses per m³
tbas=rows[44][1] #whether there is a basic tariff (y or n)
if tbas.upper()=='Y': #if there is a basic tariff, the program reads its value
    tbasicw=Decimal(rows[44][2])
    tbasics=Decimal(rows[44][3])
else:
    tbasicw=tbasics=0
tmin=Decimal(rows[44][4])#min water tariff for the first consumption range
tariff=Decimal(rows[40][3]) #rate of tariff increase per year
totalexpenses=Decimal(rows[40][4]) #rate of total expense increase per year
discount=Decimal(rows[40][5]) #rate of discount increase per year
costs=Decimal(rows[40][6]) #rate of costs increase per year
connections=Decimal(rows[40][7]) #rate of connections increase per year

###Make a list of consumption ranges and another with the respective tariffs
consumption_range=Decimal(rows[44][0]) #number of consumption ranges
cr=0 #consumption ranges counter
l_v=[] #list of volumes by consumption ranges
l_t=[] #list of tariffs by consumption ranges
while cr<consumption_range*2:
    l_v.append(Decimal(rows[46][cr])) #adds the elements to the consumption ranges volumes list
    l_t.append(Decimal(rows[46][cr+1])) #adds the elements to the tariffs list
    cr+=2 

##CONSUMPTION DATA
###Create the average monthly and daily consumption list
l_monthlyconsumption=[] #list of average monthly consumption for all consumption ranges
l_dailyconsumption=[] #list of average daily consumption for all consumption ranges
counter_monthlyconsumption=0 #consumption ranges counter
while counter_monthlyconsumption<consumption_range:
    l_mc=[] #list of average monthly consumption for the building
    l_dc=[] #list of average daily consumption for the building
    x=1 #monthly consumption counter
    while x<=12:
        l_mc.append(Decimal(rows[56+counter_monthlyconsumption][x])) #adds the elements in the monthly consumption list
        l_dc.append(Decimal(rows[67+counter_monthlyconsumption][x])) #adds the elements in the daily consumption list
        x+=1
    counter_monthlyconsumption+=1 #monthly consumption counter
    l_monthlyconsumption.append(l_mc) #appends monthly consumption of each consumption ranges
    l_dailyconsumption.append(l_dc) #appends daily consumption of each consumption ranges

##RAINFALL DATA
###Open file
file_rainfall=('Rain_Serrolandia') #rainfall file name (data in mm) - the number of years must be the same than in each scenario 
rainfall=[] #list of precipitation data
with open(file_rainfall+'.csv',newline='') as file1: #open the file and turn each line into a list
    file2=(csv.reader(file1,delimiter=';'))
    for row1 in file2:
        file3=(csv.reader(row1,delimiter=','))
        l_year=[] #list precipitation data
        for row2 in file3:
            file4=(csv.reader(row2,delimiter=','))
            for row3 in file4:
                l_year.extend(list(row3))
        rainfall.append(l_year)

###Calculate precipitation series size in years
enddate=date0+relativedelta(years=lifespan) #end date = start + series size   
qtdays=(enddate-date0) #qty of days
lifespan_calculated=Decimal((qtdays.days)/365) #days to years
    
##CONFIGURATION OF RWHS
percent=0.53
###Make a list of tank volumes and costs per users groups
user_group=2 #users considered, it can be read from file: int(rows[2][1])
tankperuser_counter=1 #tanks counter
l_tankperuser=[] #tank per consumption range
l_costsres=[] #cost per volume
while tankperuser_counter<2*consumption_range: #while the counter is less than the considered qty
    l_tankperuser.append(Decimal(rows[2+2*user_group][tankperuser_counter])) #make a list with the tank volumes per consumption range
    l_costsres.append(Decimal(rows[2+2*user_group][tankperuser_counter+1])) #make a list with the cost per volume
    tankperuser_counter+=2 
###Make a list of tariff policy
tariff_policy=3 #tariff policy considered, it can be read from file: int(rows[11][1])  
percentage_sewage=Decimal(rows[11+2*tariff_policy][1]) #percentage of sewage (%)
charge_sewage=(rows[11+2*tariff_policy][2]) #sewage volume unbilled is charged? (%)
t_ext=rows[11+2*tariff_policy][3] #whether a extra tariff is charged (y or n)
vol_t_ext=[] #list of extra volume charged by consumption range
if percentage_sewage==0:
    tbasics=0
cr=0
while cr<consumption_range:
    vol_t_ext.append(Decimal(rows[11+2*tariff_policy][4+cr])) #add the elements to the list of extra volume charged per consumption range
    cr+=1

#OPEN RESULTS FILE
with open ('Results-Serrolandia14.csv','w',encoding="utf-8") as file: #first line of the results file
    file.write('Rainfall (mm/year)'+','+'Consumption without RH (m³/year)'+','+'Consumption RH (m³/year)'+','+'Total consumption (m³/year)'+\
               ','+'Operating income - total ($)'+','+'Total expenses ($)'+','+'Financial performance'+\
               ','+'Users cons range 1'+','+'Users last cons range'+','+'Revenues - Users cons range 1'+','+'Revenues - users last cons range'+\
               ','+'Cons RH cr1 (m³)'+','+'Cons RH cr2 (m³)'+','+'Cons RH cr3 (m³)'+','+'Cons RH cr4 (m³)'+\
               ','+'Cons RH cr5 (m³)'+','+'Cons RH cr6 (m³)'+','+'Cons RH cr7 (m³)'+','+'Cons RH cr8 (m³)'+\
               ','+'Cons RH jan (m³)'+','+'Cons RH feb (m³)'+','+'Cons RH mar (m³)'+','+'Cons RH apr (m³)'+\
               ','+'Cons RH may (m³)'+','+'Cons RH jun (m³)'+','+'Cons RH jul (m³)'+','+'Cons RH aug (m³)'+\
               ','+'Cons RH sep (m³)'+','+'Cons RH oct (m³)'+','+'Cons RH nov (m³)'+','+'Cons RH dec (m³)'+\
               ','+'Cons cr1 (m³)'+','+'Cons cr2 (m³)'+','+'Cons cr3 (m³)'+','+'Cons cr4 (m³)'+\
               ','+'Cons cr5 (m³)'+','+'Cons cr6 (m³)'+','+'Cons cr7 (m³)'+','+'Cons cr8 (m³)'+\
               ','+'Cons jan (m³)'+','+'Cons feb (m³)'+','+'Cons mar (m³)'+','+'Cons apr (m³)'+\
               ','+'Cons may (m³)'+','+'Cons jun (m³)'+','+'Cons jul (m³)'+','+'Cons aug (m³)'+\
               ','+'Cons sep (m³)'+','+'Cons oct (m³)'+','+'Cons nov (m³)'+','+'Cons dec (m³)'+\
               ','+'Exp cr1 ($)'+','+'Exp cr2 ($)'+','+'Exp cr3 ($)'+','+'Exp cr4 ($)'+\
               ','+'Exp cr5 ($)'+','+'Exp cr6 ($)'+','+'Exp cr7 ($)'+','+'Exp cr8 ($)'+\
               ','+'Exp jan ($)'+','+'Exp feb ($)'+','+'Exp mar ($)'+','+'Exp apr ($)'+','+'Exp may ($)'+','+'Exp jun ($)'+\
               ','+'Exp jul ($)'+','+'Exp aug ($)'+','+'Exp sep ($)'+','+'Exp oct ($)'+','+'Exp nov ($)'+','+'Exp dec ($)'+\
               ','+'Rev cr1 ($)'+','+'Rev cr2 ($)'+','+'Rev cr3 ($)'+','+'Rev cr4 ($)'+\
               ','+'Rev cr5 ($)'+','+'Rev cr6 ($)'+','+'Rev cr7 ($)'+','+'Rev cr8 ($)'+\
               ','+'Rev jan ($)'+','+'Rev feb ($)'+','+'Rev mar ($)'+','+'Rev apr ($)'+','+'Rev may ($)'+','+'Rev jun ($)'+\
               ','+'Rev jul ($)'+','+'Rev aug ($)'+','+'Rev sep ($)'+','+'Rev oct ($)'+','+'Rev nov ($)'+','+'Rev dec ($)'+\
               ','+'FI cr1'+','+'FI cr2'+','+'FI cr3'+','+'FI cr4'+','+'FI cr5'+','+'FI cr6'+','+'FI cr7'+','+'FI cr8'+\
               ','+'FI jan'+','+'FI feb'+','+'FI mar'+','+'FI apr'+','+'FI may'+','+'FI jun'+\
               ','+'FI jul'+','+'FI aug'+','+'FI sep'+','+'FI oct'+','+'FI nov'+','+'FI dec'+\
               ','+'Cons RH cr1 - user (m³)'+','+'Cons RH cr2 - user (m³)'+','+'Cons RH cr3 - user (m³)'+','+'Cons RH cr4 - user (m³)'+\
               ','+'Cons RH cr5 - user (m³)'+','+'Cons RH cr6 - user (m³)'+','+'Cons RH cr7 - user (m³)'+','+'Cons RH cr8 - user (m³)'+\
               ','+'Cons cr1 - user (m³)'+','+'Cons cr2 - user (m³)'+','+'Cons cr3 - user (m³)'+','+'Cons cr4 - user (m³)'+\
               ','+'Cons cr5 - user (m³)'+','+'Cons cr6 - user (m³)'+','+'Cons cr7 - user (m³)'+','+'Cons cr8 - user (m³)'+\
               ','+'Exp cr1 - user ($)'+','+'Exp cr2 - user ($)'+','+'Exp cr3 - user ($)'+','+'Exp cr4 - user ($)'+\
               ','+'Exp cr5 - user ($)'+','+'Exp cr6 - user ($)'+','+'Exp cr7 - user ($)'+','+'Exp cr8 - user ($)'+\
               ','+'Rev cr1 - user ($)'+','+'Rev cr2 - user ($)'+','+'Rev cr3 - user ($)'+','+'Rev cr4 - user ($)'+\
               ','+'Rev cr5 - user ($)'+','+'Rev cr6 - user ($)'+','+'Rev cr7 - user ($)'+','+'Rev cr8 - user ($)'+\
               ','+'Exp reduct cr1 ($)'+','+'Exp reduct cr2 ($)'+','+'Exp reduct cr3 ($)'+','+'Exp reduct cr4 ($)'+\
               ','+'Exp reduct cr5 ($)'+','+'Exp reduct cr6 ($)'+','+'Exp reduct cr7 ($)'+','+'Exp reduct cr8 ($)'+\
               ','+'Rev reduct cr1 ($)'+','+'Rev reduct cr2 ($)'+','+'Rev reduct cr3 ($)'+','+'Rev reduct cr4 ($)'+\
               ','+'Rev reduct cr5 ($)'+','+'Rev reduct cr6 ($)'+','+'Rev reduct cr7 ($)'+','+'Rev reduct cr8 ($)'+\
               ','+'SD cr1 (%)'+','+'SD cr2 (%)'+','+'SD cr3 (%)'+','+'SD cr4 (%)'+\
               ','+'SD cr5 (%)'+','+'SD cr6 (%)'+','+'SD cr7 (%)'+','+'SD cr8 (%)'+\
               ','+'REL cr1 (%)'+','+'REL cr2 (%)'+','+'REL cr3 (%)'+','+'REL cr4 (%)'+\
               ','+'REL cr5 (%)'+','+'REL cr6 (%)'+','+'REL cr7 (%)'+','+'REL cr8 (%)'+\
               ','+'NPV cr1 ($)'+','+'NPV cr2 ($)'+','+'NPV cr3 ($)'+','+'NPV cr4 ($)'+\
               ','+'NPV cr5 ($)'+','+'NPV cr6 ($)'+','+'NPV cr7 ($)'+','+'NPV cr8 ($)'+\
               ','+'CRH1 (m³)'+','+'CRH2 (m³)'+','+'CRH3 (m³)'+','+'CRH4 (m³)'+','+'CRH5 (m³)'+','+'CRH6 (m³)'+','+'CRH7 (m³)'+','+'CRH8 (m³)'+\
               ','+'CRH9 (m³)'+','+'CRH10 (m³)'+','+'CRH11 (m³)'+','+'CRH12 (m³)'+','+'CRH13 (m³)'+','+'CRH14 (m³)'+','+'CRH15 (m³)'+','+'CRH16 (m³)'+\
               ','+'CRH17 (m³)'+','+'CRH18 (m³)'+','+'CRH19 (m³)'+','+'CRH20 (m³)'+','+'CRH21 (m³)'+','+'CRH22 (m³)'+','+'CRH23 (m³)'+\
               ','+'CRH24 (m³)'+','+'CRH25 (m³)'+','+'CRH26 (m³)'+','+'CRH27 (m³)'+','+'CRH28 (m³)'+','+'CRH29 (m³)'+','+'CRH30 (m³)'+\
               ','+'C1 (m³)'+','+'C2 (m³)'+','+'C3 (m³)'+','+'C4 (m³)'+','+'C5 (m³)'+','+'C6 (m³)'+','+'C7 (m³)'+','+'C8 (m³)'+\
               ','+'C9 (m³)'+','+'C10 (m³)'+','+'C11 (m³)'+','+'C12 (m³)'+','+'C13 (m³)'+','+'C14 (m³)'+','+'C15 (m³)'+','+'C16 (m³)'+\
               ','+'C17 (m³)'+','+'C18 (m³)'+','+'C19 (m³)'+','+'C20 (m³)'+','+'C21 (m³)'+','+'C22 (m³)'+','+'C23 (m³)'+\
               ','+'C24 (m³)'+','+'C25 (m³)'+','+'C26 (m³)'+','+'C27 (m³)'+','+'C28 (m³)'+','+'C29 (m³)'+','+'C30 (m³)'+'\n')
#SIZING
lr=[] #list of reservoirs
lcm_rh_c=[] #monthly consumption of RH list for all consumption range
lcm_c=[] #monthly consumption list for all consumption range
lexpm_c=[] #monthly expenses list for all consumption range
lrevm_c=[] #monthly revenues list for all consumption range
lcy_norh_c=[] #yearly consumption with no RH list for all consumption range
lcy_rh_c=[] #yearly consumption of RH list for all consumption range
lcy_c=[] #yearly consumption list for all consumption range
lexpy_c=[] #yearly expenses list for all consumption range
lrevy_c=[] #yearly revenues list for all consumption range
lcy_rh_c_des=[] #yearly consumption of RH list per connection for all consumption range
lcy_c_des=[] #yearly consumption list per connection for all consumption range
lexpy_c_des=[] #yearly expenses list per connection for all consumption range
lexpy_c_reduction=[] #yearly reduction of expenses list per connection for all consumption range
lrevy_c_des=[] #yearly revenues of RH list per connection for all consumption range
lrevy_c_reduction=[] #yearly reduction of revenues list per connection for all consumption range
lcm_rh_city=[] #monthly RH consumption for all the city
lcm_city=[] #monthly consumption for all the city
lcy_ev_c=[] #consumption with RH in each year of each consumption range list
lcy_norh_ev_c=[] #consumption with no RH in each year of each consumption range list
lev=[] #consumption with RH in each year of the city list
lev_norh=[] #consumption with RH in each year of the city list
lexpm_city=[] #monthly expenses for all the city
lrevm_city=[] #monthly revenues for all the city
lcy_rh_city=[] #yearly RH consumption for all the city
lcy_city=[] #yearly consumption for all the city
lexpy_city=[] #yearly expenses for all the city
lrevy_city=[] #yearly revenues for all the city
cy_rh_c=0 #average RH consumption per year of rainwater for each connection
cy_rh_c_des=0 #average RH consumption per year of rainwater for each connection 
cy_c=0 #average consumption per year of rainwater for each connection
cy_c_des=0 #average consumption per year of rainwater for each connection 
cr=0 #consumption range initial
lsd=[] #satisfied demand list - demand met
lct=[] #total consumption list
lve=[] #overflow volume list
lrh=[] #list of consumption efficiency in relation to available volume
lrel=[] #reliability list
lnpv=[] #npv list

##Evaluation for each consumption range
for (res,connect,area,demand,vol_ext,costinitial) in zip (l_tankperuser, l_connections, l_areas, l_dailyconsumption, vol_t_ext, l_costsres): #for a reservoir volume in the volume list
    v0=0 #volume in reservoir in the beginning
    dt=0 #total demand
    ct=0 #total consumption
    qt=0 #total available volume
    drel=0 #days of reliability - days that meets the demand completely
    date=date0 #date
    preciptotal=0 #volume of rain
    l_lpreci=0 #counter of the days in rain list
    d10=d20=d30=Decimal(0) #consumptions on days 1, 2 and 3
    lcm_norh=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly consumption with no RH for each consumption range        
    lcm_rh=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly consumption of RH for each consumption range
    lcm_rh_des=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly consumption of RH for each connection
    lcm=[0,0,0,0,0,0,0,0,0,0,0,0]  #total monthly consumption for each consumption range
    lcm_des=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly consumption for each connection
    lrev=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly revenues for each consumption range
    lrev_des=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly revenues for each connection
    lrev_reduction=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly revenues reduction for each connection
    lexp=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly expenses for each consumption range
    lexp_des=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly expenses for each connection
    lexp_reduction=[0,0,0,0,0,0,0,0,0,0,0,0] #monthly expenses reduction for each connection
    lcy_ev=[] #consumption with RH in each year of each consumption range
    lcy_norh_ev=[] #consumption with no RH in each year of each consumption range
    cr+=1 #consumption ranges counter
    txtarifftotal=Decimal(tariff) #initial tariff (input data)
    txexpensestotal=Decimal(totalexpenses) #initial expense (input data)
    txconnectionstotal=Decimal(connections) #initial connections number (input data)
    txdiscounttotal=Decimal(discount) #initial discount rate per month (input data)
    txcoststotal=Decimal(costs) #initial costs (input data)
    year_=0 #initial value of the year
    cons_m=0 #initial value for average monthly consumption with RH
    cons_rh_m=0 #initial value for average monthly RH consumption
    cons_norh_m=0  #initial value for average monthly consumption with no RH
    cons_y=0 #initial value for average yearly consumption with RH 
    cons_norh_y=0 #initial value for average yearly consumption with no RH
    dr_month=1 #initial discount rate per month
    expenses=0 #initial expense
    expenses_norh=0 #initial expense with no RH
    costm_vp=revenues_vp=vacum=0    

    #Hydric balance
    for (j) in (rainfall[0]): #sizing for each day of precipitation
        con=ceil((Decimal(connect))*Decimal(txconnectionstotal)) #updates the number of connections
        expense=expen_m*txexpensestotal #updates the expense
        dr_year=Decimal(td)*Decimal(txdiscounttotal) #updates the discount rate
        preciptotal+=Decimal(j) #sum the rainfall of each day
        l_lpreci+=1 #counts the number of days in the rainfall list
        month_=date.month #looks the month to calculate rainwater consumption per month   
        k=demand[month_-1] #daily consumption of each month   
        if (Decimal(d10)<Decimal(1)) and (Decimal(d20)<Decimal(1)) and (Decimal(d30)<Decimal(1)):  
            discard_i=discard_ini #if the precipitation on days 1, 2 and 3 is less than 1 mm, discards its volume
        else:
            discard_i=Decimal(0) #if the precipitation on days 1, 2 and 3 is bigger than 1 mm, the discard is 0
        d30=d20
        d20=d10
        d10=j
        q=(Decimal(j)-discard_i)*area*c_runoff/1000 #volume available per day in m³ (precipitation - discard)
        if q<=0: #if q is less than 0 it is zero
            q=0
        vc=min((Decimal(k)/1000),((teta*Decimal(q))+v0)) #volume consumed = demand or volume of the day + vol of the previous day
        if vc>=Decimal(k)/1000: #if demand is consumed, adds this day to the days of reliability
            drel+=1 #days of reliability
        v=min((q+v0-vc),(res-((1-teta)*vc))) #volume available for the next day
        v0=v  #volume in reservoir is now v
        dt+=(Decimal(k)/Decimal(percent))/1000 #total demand
        ct+=vc #total consumption
        qt+=q #total rainwater volume available
        lcm_norh[month_-1]+=(((Decimal(k)/Decimal(percent))/1000)*con)/lifespan #sum the daily consumption of each month and divide by the qty of years - average monthly consumption with no RH
        lcm_rh[month_-1]+=(vc*con)/lifespan #sum the daily consumption of each month and divide by the qty of years - average RH monthly consumption per consumption range
        lcm_rh_des[month_-1]+=(vc)/lifespan #sum the daily consumption of each month and divide by the qty of years - average RH monthly consumption per connection
        lcm[month_-1]+=((((Decimal(k)/Decimal(percent))/1000)-vc)*con)/lifespan #sum the daily consumption of each month and divide by the qty of years - average monthly consumption per consumption range
        lcm_des[month_-1]+=((((Decimal(k)/Decimal(percent))/1000)-vc))/lifespan #sum the daily consumption of each month and divide by the qty of years - average monthly consumption per connection
        expenses+=((((Decimal(k)/Decimal(percent))/1000)-vc)*expense*con)/lifespan #sum the daily expense of each month and divide by the qty of years - average monthly expenses per consumption range
        expenses_norh+=((Decimal(k)/Decimal(percent)/1000)*expense*con)/lifespan #sum the daily expense of each month and divide by the qty of years - average monthly expenses per connection
        cons_m+=(((Decimal(k)/Decimal(percent))/1000)-vc) #average monthly consumption with RH per connection - sum the daily consumption of each month
        cons_norh_y+=((Decimal(k)/Decimal(percent))/1000)*con #average yearly consumption with no RH per consumption range - sum the daily consumption of each day
        cons_y+=(((Decimal(k)/Decimal(percent))/1000)-vc)*con #average yearly consumption with RH per consumption range - sum the daily consumption of each day
        cons_norh_m+=Decimal(k)/Decimal(percent)/1000 #average monthly consumption with no RH per consumption range - sum the daily consumption of each day
        cons_rh_m+=vc #average monthly RH consumption per consumption range - sum the daily consumption of each day  
        date=date+timedelta(days=1) #updates the date - sum one day
        datenextyear=date0+relativedelta(years=year_+1) #defines the date that the year begins
        
        if date.day==1 or date==enddate: #if it is day 1 of some month or the date is equal the end date
            dr_month*=(1+(dr_year/100)) #updates the discount rate for the month
            lexp[month_-1]+=(expenses)/(dr_month) #calculates the present value of expenses per consumption range
            lexp_des[month_-1]+=(expenses/con)/(dr_month) #calculates the present value of expenses per connection
            lexp_reduction[month_-1]+=((expenses_norh-expenses)/(dr_month)) #calculates the present value of expenses with no RH per connection
            expenses=0 #the expense in the next month is 0
            expenses_norh=0 #the expense with no RH in the next month is 0
            e1=e2=e7=0 #volume already quantified in the water tariff calculation
            p1=p2=p1_norh=0 #price of the water volume consumed in the consumption range
            if res==0:
                vol_ext=0                    
            consumption=max((cons_m+vol_ext),tmin) #adds the extra volume to the real consumption 

                
            #Tariff of water and sewage per month with RH   
            for f,g in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
                if consumption>f: #there are more consumption ranges to consider
                    p1+=((f-e1)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                else: #there are no more consumption ranges to consider
                    p1+=((consumption-e1)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                    break
                e1=f #to remove the volume already quantified in the previous track
            if charge_sewage.upper()=='N': #considering that: water consumption = sewage consumption
                p2=p1
            else: #considering that: water consumption is lower than sewage consumption
                consumption=max((cons_norh_m+vol_ext),tmin)
                for f,g in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
                    if consumption>f: #there are more consumption ranges to consider
                        p2+=((f-e2)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                    else: #there are no more consumption ranges to consider
                        p2+=((consumption-e2)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                        break
                    e2=f #to remove the volume already quantified in the previous track
            #Tariff of water and sewage per month with no RH                   
            for f,g in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
                consumption=max((cons_norh_m),tmin)
                if consumption>f: #there are more consumption ranges to consider
                    p1_norh+=((f-e7)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                else: #there are no more consumption ranges to consider
                    p1_norh+=((consumption-e7)*g) #price: volume consumed in the consumption range times the cost/m³ of this range
                    break
                e7=f #to remove the volume already quantified in the previous track     

            revenues=(tbasicw+tbasics+p1+p2*(percentage_sewage/100))*Decimal(txtarifftotal) #price*(% sewage)*(annual water increase rate)
            revenues_norh=(tbasicw+tbasics+p1_norh*(1+(percentage_sewage/100)))*Decimal(txtarifftotal) #price*(% sewage)*(annual water increase rate)
            revenues_user=(revenues_norh-revenues)/(dr_month) #revenues for user: tariff with no RH - tariff with RH
            lrev[month_-1]+=(revenues*con)/(lifespan*dr_month) #calculates the present value of revenues per consumption range
            lrev_des[month_-1]+=(revenues)/(lifespan*dr_month) #calculates the present value of revenuess per connection
            lrev_reduction[month_-1]+=((revenues_norh-revenues)*con)/(lifespan*dr_month) #calculates the reduction in revenues with RH implantation for the company per consumption range

            #Calculate the expenses for users
            if month_==1 or month_==2 or month_==4 or month_==5 or month_==7 or month_==8 or month_==10 or month_==11: #to calculate costs for different months
                costm=cmonthlyf+cmonthlyv*cons_rh_m #months that only have a monthly cost
            elif month_==3 or month_==9: #months with monthly and quarterly cost
                costm=cmonthlyf+cmonthlyv*cons_rh_m+cquarterly
            elif month_==6: #month with monthly, quarterly and half-yearly cost
                costm=cmonthlyf+cmonthlyv*cons_rh_m+cquarterly+csemester
            elif month_==12: #month with monthly, quarterly, semi-annual and annual cost
                costm=cmonthlyf+cmonthlyv*cons_rh_m+cquarterly+csemester+cannualf+cannualv*costinitial
             
            if res==0:
                costm=0
                
            costm_du=(costm*txcoststotal)/(dr_month) #value present of monthly cost                 
            costm_vp+=costm_du #accumulated value present of monthly cost     
            revenues_vp+=revenues_user #accumulated value present of monthly revenues for user   
            vp=revenues_user-costm_du #present value monthly for user
            vacum+=vp #accumulated present value for user
            cons_m=0 #the consumption with RH in the next month is 0
            cons_rh_m=0 #the RH consumption in the next month is 0
            cons_norh_m=0 #the consumption with no RH in the next month is 0

            if date==datenextyear: #if it is day 1 of the year
                lcy_ev.append(cons_y) #adds to list the consumption with RH in the year per consumption range
                cons_y=0 #the consumption with RH in the next year=0
                lcy_norh_ev.append(cons_norh_y) #adds to list the consumption with no RH in the year per consumption range
                cons_norh_y=0 #the consumption with no RH in the next year=0
                    
        if l_lpreci==len(rainfall[0]) or date==enddate: #if it is the last day
            break

        if date==datenextyear: #if it is day 1 of the year
            year_+=1
            if year_<lifespan:
                txtarifftotal*=Decimal(tariff) #updates the water tariff increase value
                txexpensestotal*=Decimal(totalexpenses) #updates the cost rate increase value
                txconnectionstotal*=Decimal(connections) #updates the connections increase amount in the year  
                txdiscounttotal*=Decimal(discount) #updates the discount rate increase value
                txcoststotal*=Decimal(costs) #updates the discount rate increase value
    
    cy_norh_c=sum(lcm_norh) #consumption with no RH of the consumption range
    cy_rh_c=sum(lcm_rh) #RH consumption of the consumption range
    cy_c=sum(lcm) #consumption of the consumption range
    exp_c=sum(lexp) #expense of the consumption range
    rev_c=sum(lrev) #revenue of the consumption range
    cy_rh_c_des=sum(lcm_rh_des) #RH consumption of the connection
    cy_c_des=sum(lcm_des) #consumption of the connection
    exp_c_des=sum(lexp_des) #expense of the connection
    rev_c_des=sum(lrev_des) #revenue of the connection
    exp_c_reduction=sum(lexp_reduction) #reduction of expense of the connection
    rev_c_reduction=sum(lrev_reduction) #reduction of revenue of the connection
    lcy_norh_c.append(cy_norh_c) #yearly consumption with no RH list for consumption range
    lcy_rh_c.append(cy_rh_c) #yearly RH consumption list for consumption range
    lcy_c.append(cy_c) #yearly consumption list for consumption range
    lcy_ev_c.append(lcy_ev) #add to list the consumption with RH in each year of each consumption range
    lcy_norh_ev_c.append(lcy_norh_ev) #add to list the consumption with RH in each year of each consumption range 
    lexpy_c.append(exp_c) #yearly expense list for consumption range
    lrevy_c.append(rev_c) #yearly revenue list for consumption range
    lcy_rh_c_des.append(cy_rh_c_des) #yearly RH consumption list for connection
    lcy_c_des.append(cy_c_des) #yearly consumption list for connection
    lexpy_c_des.append(exp_c_des) #yearly expense list for connection
    lrevy_c_des.append(rev_c_des) #yearly revenue list for connection
    lexpy_c_reduction.append(exp_c_reduction) #yearly expense reduction list for connection
    lrevy_c_reduction.append(rev_c_reduction) #yearly revenue reduction list for connection
    lr.append(res) #list of reservoirs
    lcm_rh_c.append(lcm_rh) #average monthly RH consumption per year of rainwater for all consumption range
    lcm_c.append(lcm) #average monthly consumption per year of rainwater for all consumption range
    lexpm_c.append(lexp) #average monthly expense per year of rainwater for all consumption range
    lrevm_c.append(lrev) #average monthly revenue per year of rainwater for all consumption range
                               
    ###Indicators for users of RW systems
    lsd.append((ct/dt)*100) #satisfied demand list - demand met
    lct.append(ct) #total consumption list
    lve.append(qt-ct) #overflow volume list
    lrh.append((ct/qt)*100) #list of consumption efficiency in relation to available volume
    lrel.append((drel/l_lpreci)*100) #reliability list
    lnpv.append(vacum-costinitial) #npv list

rainannual=preciptotal/lifespan #calculates the rainfall per year
cy_norh_city=sum(lcy_norh_c) #yearly consumption with no RH for the city
cy_rh_city=sum(lcy_rh_c) #yearly RH consumption for the city
cy_city=sum(lcy_c) #yearly consumption for the city
expy_city=sum(lexpy_c) #yearly expense for the city
revy_city=sum(lrevy_c) #yearly revenue for the city
if_city=revy_city/expy_city #yearly financial indicator for the city
    
###Quantify the users who could be supplied with the volume saved
demand_users1=(l_monthlyconsumption[0]) #demand of the users of the consumtion range 1
tc=int(total_connections-1)
demand_users_last=(l_monthlyconsumption[tc]) #demand of the users of the last consumtion range 
e3=e4=0
p3=p4=0
demand_considered=max(tmin,demand_users1[0])
for f,g in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
    if (demand_considered)>f: #there are more consumption ranges to consider
        p3+=(f-e3)*g #volume consumed in the consumption range times the cost/m³ of this range
    else: #there are no more consumption ranges to consider
        p3+=(demand_users1[0]-e3)*g #volume consumed in the consumption range times the cost/m³ of this range
        break
    e3=f #to remove the volume already quantified in the previous track                
for ff,gg in zip (l_v,l_t): #consumption ranges and prices for the total volume consumed
    if demand_users_last[0]>ff: #there are more consumption ranges to consider
        p4+=(ff-e4)*gg #volume consumed in the consumption range times the cost/m³ of this range
    else: #there are no more consumption ranges to consider
        p4+=(demand_users_last[0]-e4)*gg #volume consumed in the consumption range times the cost/m³ of this range
        break
    e4=ff #to remove the volume already quantified in the previous track       
users_1=round((cy_rh_city/(12*demand_users1[0]))-Decimal(0.5)) #number of users of the consumption range 1 that can be be supplied (roud down)
users_last=round((cy_rh_city/(12*demand_users_last[0]))-Decimal(0.5)) #number of users of the consumption range 8 that can be be supplied (roud down)
revusers_1=users_1*(12*((1+(percentage_sewage/100))*p3+tbasicw+tbasics)) #monthly revenue if users of the consumption range 1 were supplied
revusers_last=users_last*(12*((1+(percentage_sewage/100))*p4+tbasicw+tbasics)) #monthly revenue if users of the consumption range 8 were supplied

###Calculate the indicators for the city
lcy_rh_city.append(cy_rh_city) #yearly RH consumption for all the city
lcy_city.append(cy_city) #yearly consumption for all the city
lexpy_city.append(expy_city) #yearly expenses for all the city
lrevy_city.append(revy_city) #yearly revenues for all the city

while total_connections<8:
    lcy_c.append(0)
    lcy_c_des.append(0)
    lcy_rh_c.append(0)
    lcy_rh_c_des.append(0)
    lexpy_c.append(0)
    lexpy_c_des.append(0)
    lrevy_c.append(0)
    lrevy_c_des.append(0) 
    lexpy_c_reduction.append(0)
    lrevy_c_reduction.append(0)   
    lsd.append(0)
    lrel.append(0)
    lnpv.append(0)
    lcm_rh_c.append([0,0,0,0,0,0,0,0,0,0,0,0])
    lcm_c.append([0,0,0,0,0,0,0,0,0,0,0,0])
    lexpm_c.append([0,0,0,0,0,0,0,0,0,0,0,0])
    lrevm_c.append([0,0,0,0,0,0,0,0,0,0,0,0])
    total_connections+=1
    
for w0, w1, w2, w3, w4, w5, w6, w7 in zip(lcm_rh_c[0],lcm_rh_c[1],lcm_rh_c[2],lcm_rh_c[3],lcm_rh_c[4],lcm_rh_c[5],lcm_rh_c[6],lcm_rh_c[7]):
    lcm_rh_city.append(w0+w1+w2+w3+w4+w5+w6+w7) #average monthly RH consumption for the city
for w0, w1, w2, w3, w4, w5, w6, w7 in zip(lcm_c[0],lcm_c[1],lcm_c[2],lcm_c[3],lcm_c[4],lcm_c[5],lcm_c[6],lcm_c[7]):
    lcm_city.append(w0+w1+w2+w3+w4+w5+w6+w7) #average monthly consumption for the city
for w0, w1, w2, w3, w4, w5, w6, w7 in zip(lexpm_c[0],lexpm_c[1],lexpm_c[2],lexpm_c[3],lexpm_c[4],lexpm_c[5],lexpm_c[6],lexpm_c[7]):
    lexpm_city.append(w0+w1+w2+w3+w4+w5+w6+w7) #average monthly expenses for the city
for w0, w1, w2, w3, w4, w5, w6, w7 in zip(lrevm_c[0],lrevm_c[1],lrevm_c[2],lrevm_c[3],lrevm_c[4],lrevm_c[5],lrevm_c[6],lrevm_c[7]):
    lrevm_city.append(w0+w1+w2+w3+w4+w5+w6+w7) #average monthly revenues for the city

###Calculate the consumption evolution - consumption of the city for each year of the lifespan
lcy_ev_c=list(zip(*lcy_ev_c)) #reverses the lists: consumption ranges by year for year by consumption range
for i in lcy_ev_c: #for each year       
    lev.append(float(sum(i))) #sum the consumption of all consumption ranges
lcy_norh_ev_c=list(zip(*lcy_norh_ev_c)) #reverses the lists: consumption ranges by year for year by consumption range
for i in lcy_norh_ev_c: #for each year         
    lev_norh.append(float(sum(i))) #sum the consumption of all consumption ranges
    
###Calculate financial indicator - Benefit Costa Rate (BCR)
lbcry_c=[] #bcr per year for each consumption range
lbcrm_city=[] #bcr for each month for the city
for n,m in zip (lrevy_c,lexpy_c): #for the revenues and expenses per year of each consumption range
    if m==0:
        lbcry_c.append(0)
    else:
        lbcry_c.append(n/m) 
for n,m in zip (lrevm_city,lexpm_city): #for the revenues and expenses per month of the city
    if m==0:
        lbcrm_city.append(0)
    else:
        lbcrm_city.append(n/m)

tx1=1+(((float(txtarifftotal))**(1/float(lifespan)))-1) #updates the water tariff value
tx2=1+(((float(txexpensestotal))**(1/float(lifespan)))-1) #updates the cost rate value
tx3=1+(((float(txconnectionstotal))**(1/float(lifespan)))-1) #updates the connections value in the month
tx4=1+(((float(txdiscounttotal))**(1/float(lifespan)))-1) #updates the discount rate value in the month
tx5=1+(((float(txcoststotal))**(1/float(lifespan)))-1) #updates the conts value in the month

        
#SAVE RESULTS TO THE FILE
with open ('Results-Serrolandia13.csv','a') as file: #save each scenario results to file
    file.write(str(rainannual)+','+str(cy_norh_city)+','+str(cy_rh_city)+','+str(cy_city)+','+str(revy_city)+\
               ','+str(expy_city)+','+str(if_city)+','+str(users_1)+','+str(users_last)+\
               ','+str(revusers_1)+','+str(revusers_last)+\
               ','+str(lcy_rh_c[0])+','+str(lcy_rh_c[1])+','+str(lcy_rh_c[2])+','+str(lcy_rh_c[3])+','+str(lcy_rh_c[4])+\
               ','+str(lcy_rh_c[5])+','+str(lcy_rh_c[6])+','+str(lcy_rh_c[7])+\
               ','+str(lcm_rh_city[0])+','+str(lcm_rh_city[1])+','+str(lcm_rh_city[2])+','+str(lcm_rh_city[3])+\
               ','+str(lcm_rh_city[4])+','+str(lcm_rh_city[5])+','+str(lcm_rh_city[6])+','+str(lcm_rh_city[7])+\
               ','+str(lcm_rh_city[8])+','+str(lcm_rh_city[9])+','+str(lcm_rh_city[10])+','+str(lcm_rh_city[11])+\
               ','+str(lcy_c[0])+','+str(lcy_c[1])+','+str(lcy_c[2])+','+str(lcy_c[3])+','+str(lcy_c[4])+\
               ','+str(lcy_c[5])+','+str(lcy_c[6])+','+str(lcy_c[7])+\
               ','+str(lcm_city[0])+','+str(lcm_city[1])+','+str(lcm_city[2])+','+str(lcm_city[3])+\
               ','+str(lcm_city[4])+','+str(lcm_city[5])+','+str(lcm_city[6])+','+str(lcm_city[7])+\
               ','+str(lcm_city[8])+','+str(lcm_city[9])+','+str(lcm_city[10])+','+str(lcm_city[11])+\
               ','+str(lexpy_c[0])+','+str(lexpy_c[1])+','+str(lexpy_c[2])+','+str(lexpy_c[3])+','+str(lexpy_c[4])+\
               ','+str(lexpy_c[5])+','+str(lexpy_c[6])+','+str(lexpy_c[7])+\
               ','+str(lexpm_city[0])+','+str(lexpm_city[1])+','+str(lexpm_city[2])+','+str(lexpm_city[3])+\
               ','+str(lexpm_city[4])+','+str(lexpm_city[5])+','+str(lexpm_city[6])+','+str(lexpm_city[7])+\
               ','+str(lexpm_city[8])+','+str(lexpm_city[9])+','+str(lexpm_city[10])+','+str(lexpm_city[11])+\
               ','+str(lrevy_c[0])+','+str(lrevy_c[1])+','+str(lrevy_c[2])+','+str(lrevy_c[3])+','+str(lrevy_c[4])+\
               ','+str(lrevy_c[5])+','+str(lrevy_c[6])+','+str(lrevy_c[7])+\
               ','+str(lrevm_city[0])+','+str(lrevm_city[1])+','+str(lrevm_city[2])+','+str(lrevm_city[3])+\
               ','+str(lrevm_city[4])+','+str(lrevm_city[5])+','+str(lrevm_city[6])+','+str(lrevm_city[7])+\
               ','+str(lrevm_city[8])+','+str(lrevm_city[9])+','+str(lrevm_city[10])+','+str(lrevm_city[11])+
               ','+str(lbcry_c[0])+','+str(lbcry_c[1])+','+str(lbcry_c[2])+','+str(lbcry_c[3])+','+str(lbcry_c[4])+\
               ','+str(lbcry_c[5])+','+str(lbcry_c[6])+','+str(lbcry_c[7])+\
               ','+str(lbcrm_city[0])+','+str(lbcrm_city[1])+','+str(lbcrm_city[2])+','+str(lbcrm_city[3])+\
               ','+str(lbcrm_city[4])+','+str(lbcrm_city[5])+','+str(lbcrm_city[6])+','+str(lbcrm_city[7])+\
               ','+str(lbcrm_city[8])+','+str(lbcrm_city[9])+','+str(lbcrm_city[10])+','+str(lbcrm_city[11])+\
               ','+str(lcy_rh_c_des[0])+','+str(lcy_rh_c_des[1])+','+str(lcy_rh_c_des[2])+','+str(lcy_rh_c_des[3])+\
               ','+str(lcy_rh_c_des[4])+','+str(lcy_rh_c_des[5])+','+str(lcy_rh_c_des[6])+','+str(lcy_rh_c_des[7])+\
               ','+str(lcy_c_des[0])+','+str(lcy_c_des[1])+','+str(lcy_c_des[2])+','+str(lcy_c_des[3])+\
               ','+str(lcy_c_des[4])+','+str(lcy_c_des[5])+','+str(lcy_c_des[6])+','+str(lcy_c_des[7])+\
               ','+str(lexpy_c_des[0])+','+str(lexpy_c_des[1])+','+str(lexpy_c_des[2])+','+str(lexpy_c_des[3])+\
               ','+str(lexpy_c_des[4])+','+str(lexpy_c_des[5])+','+str(lexpy_c_des[6])+','+str(lexpy_c_des[7])+\
               ','+str(lrevy_c_des[0])+','+str(lrevy_c_des[1])+','+str(lrevy_c_des[2])+','+str(lrevy_c_des[3])+\
               ','+str(lrevy_c_des[4])+','+str(lrevy_c_des[5])+','+str(lrevy_c_des[6])+','+str(lrevy_c_des[7])+\
               ','+str(lexpy_c_reduction[0])+','+str(lexpy_c_reduction[1])+','+str(lexpy_c_reduction[2])+','+str(lexpy_c_reduction[3])+\
               ','+str(lexpy_c_reduction[4])+','+str(lexpy_c_reduction[5])+','+str(lexpy_c_reduction[6])+','+str(lexpy_c_reduction[7])+\
               ','+str(lrevy_c_reduction[0])+','+str(lrevy_c_reduction[1])+','+str(lrevy_c_reduction[2])+','+str(lrevy_c_reduction[3])+\
               ','+str(lrevy_c_reduction[4])+','+str(lrevy_c_reduction[5])+','+str(lrevy_c_reduction[6])+','+str(lrevy_c_reduction[7])+\
               ','+str(lsd[0])+','+str(lsd[1])+','+str(lsd[2])+','+str(lsd[3])+','+str(lsd[4])+\
               ','+str(lsd[5])+','+str(lsd[6])+','+str(lsd[7])+\
               ','+str(lrel[0])+','+str(lrel[1])+','+str(lrel[2])+','+str(lrel[3])+','+str(lrel[4])+\
               ','+str(lrel[5])+','+str(lrel[6])+','+str(lrel[7])+\
               ','+str(lnpv[0])+','+str(lnpv[1])+','+str(lnpv[2])+','+str(lnpv[3])+','+str(lnpv[4])+\
               ','+str(lnpv[5])+','+str(lnpv[6])+','+str(lnpv[7])+\
               ','+str(lev[0])+','+str(lev[1])+','+str(lev[2])+','+str(lev[3])+','+str(lev[4])+','+str(lev[5])+','+str(lev[6])+','+str(lev[7])+\
               ','+str(lev[8])+','+str(lev[9])+','+str(lev[10])+','+str(lev[11])+','+str(lev[12])+','+str(lev[13])+','+str(lev[14])+','+str(lev[15])+\
               ','+str(lev[16])+','+str(lev[17])+','+str(lev[18])+','+str(lev[19])+','+str(lev[20])+','+str(lev[21])+','+str(lev[22])+\
               ','+str(lev[23])+','+str(lev[24])+','+str(lev[25])+','+str(lev[26])+','+str(lev[27])+','+str(lev[28])+','+str(lev[29])+\
               ','+str(lev_norh[0])+','+str(lev_norh[1])+','+str(lev_norh[2])+','+str(lev_norh[3])+','+str(lev_norh[4])+','+str(lev_norh[5])+','+str(lev_norh[6])+','+str(lev_norh[7])+\
               ','+str(lev_norh[8])+','+str(lev_norh[9])+','+str(lev_norh[10])+','+str(lev_norh[11])+','+str(lev_norh[12])+','+str(lev_norh[13])+','+str(lev_norh[14])+','+str(lev_norh[15])+\
               ','+str(lev_norh[16])+','+str(lev_norh[17])+','+str(lev_norh[18])+','+str(lev_norh[19])+','+str(lev_norh[20])+','+str(lev_norh[21])+','+str(lev_norh[22])+\
               ','+str(lev_norh[23])+','+str(lev_norh[24])+','+str(lev_norh[25])+','+str(lev_norh[26])+','+str(lev_norh[27])+','+str(lev_norh[28])+','+str(lev_norh[29])+\
               ','+'\n')
print(f'END!!')

dia: 1993-11-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 19706.58853586053096559273998 despesas: 5617.140995681728498072241196
dia: 1993-12-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 19503.51975977118842157545160 despesas: 7275.855277671791250695130960
dia: 1994-01-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 19315.86446482453597868706763 despesas: 6473.820991912007179387931801
dia: 1994-02-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 19123.41719507928978705303759 despesas: 5998.575506121025605641887308
dia: 1994-03-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.250498113207546855572214909
receitas: 18713.37440439975569504454485 despesas: 6347.884640393014455318825210
dia: 1994-04-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 18744.25570779374585999781803 despesas

receitas: 25167.90361545908495080845181 despesas: 61133.53580312420190529740247
dia: 2001-12-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 25104.48889838513296368162013 despesas: 55355.39953160744677250502968
dia: 2002-01-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 24488.71655936479838213845922 despesas: 49078.87092472719768977980524
dia: 2002-02-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 24156.02752639678390749008375 despesas: 54158.41703346594123318819888
dia: 2002-03-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.250498113207546855572214909
receitas: 23551.59171321236388780751444 despesas: 50052.85529541851321872461630
dia: 2002-04-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 23504.14720003902920140352118 despesas: 49200.78703844504644845441649
dia: 2002-05-01 00:00:00 enddate: 2023-10-01 00:00:00 consum

dia: 2006-12-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 23575.92647729507745459727439 despesas: 83467.58541734261694467429397
dia: 2007-01-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 23277.35149213522448989156695 despesas: 78087.70830159616929899708382
dia: 2007-02-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 23032.55562178777359403119309 despesas: 82829.38747802274827805114327
dia: 2007-03-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.250498113207546855572214909
receitas: 22255.38064049067637719421554 despesas: 69546.81026661089658393613012
dia: 2007-04-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 22145.36651163335853744022784 despesas: 70043.48328422199246958409987
dia: 2007-05-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 21696.23026748503148229342188 despesas

dia: 2011-04-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 18390.34887521659253438943003 despesas: 84133.27636530814063828496657
dia: 2011-05-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 17967.07385424513405946005318 despesas: 88851.56710483546096212035178
dia: 2011-06-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 17690.01189030704731636197322 despesas: 90981.42322297025628424876809
dia: 2011-07-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 17282.85593015308975365571487 despesas: 82860.90691090819263896281182
dia: 2011-08-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 17131.31024889981658589960807 despesas: 85230.46539970899984302899859
dia: 2011-09-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 16689.19387731289547191767670 despesas

dia: 2016-03-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.473730188679244957556936870
receitas: 12044.37795915466445046415039 despesas: 98794.81857658752330515553405
dia: 2016-04-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 12091.54093010993849780213011 despesas: 99792.99500531026359963735166
dia: 2016-05-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 11709.19085168791197900939471 despesas: 102011.9382795580755911605269
dia: 2016-06-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 11309.56500810384909563945980 despesas: 102422.7021498018562895100336
dia: 2016-07-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 11003.42974759582675532412058 despesas: 92324.90482421794709418756843
dia: 2016-08-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 10788.81225609370751203683016 despesas

dia: 2020-08-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 6148.146534001587909998978319 despesas: 100000.1490209922912385467973
dia: 2020-09-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 5981.265808487160387202839445 despesas: 106821.8123282100655576708176
dia: 2020-10-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 5796.426090231441598545544606 despesas: 119784.0205605560717956782983
dia: 2020-11-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 6797.215312910496255364016683 despesas: 146575.4032475040433410834358
dia: 2020-12-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.696962264150943059541658831
receitas: 6484.096620209409346051702840 despesas: 124176.6712743428040266894963
dia: 2021-01-01 00:00:00 enddate: 2023-10-01 00:00:00 consumo 6.920194339622641161526380792
receitas: 6451.671953563896630236170993 despesas